### 필요 라이브러리 임포트

In [1]:
import os
import pandas as pd
from transformers import *
from summarizer import Summarizer
import numpy as np

### 데이터 불러오기 및 첫 정제

In [262]:
btsdf = pd.read_csv ('./data/news_dfv02.csv',sep ='w')
btsdf

,Unnamed: 0,media,date,title,article_original,url
0,0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...,...
866,866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


### EDA

In [263]:
btsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        871 non-null    int64 
 1   media             871 non-null    object
 2   date              871 non-null    object
 3   title             871 non-null    object
 4   article_original  858 non-null    object
 5   url               871 non-null    object
dtypes: int64(1), object(5)
memory usage: 41.0+ KB


In [264]:
btsdf = btsdf.dropna()

In [265]:
btsdf

,Unnamed: 0,media,date,title,article_original,url
0,0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...,...
866,866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [266]:
btsnews = btsdf[['date','article_original']].copy()

In [267]:
btsnews = pd.DataFrame(btsnews)

In [268]:
btsnews = btsnews.reset_index(drop=True)

In [269]:
btsnews.head(70)

,date,article_original
0,2020-01-02,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...
1,2020-01-02,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...
2,2020-01-03,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...
3,2020-01-03,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...
4,2020-01-06,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...
...,...,...
65,2020-02-11,앞서 봉 감독은 골든글로브 시상식 레드카펫 행사에서 캐나다 언론이 한국이 독창성을...
66,2020-02-11,방탄소년단은 10일 트위터에 “봉준호 감독님 정말 정말 정말 정말로 정말이지 정말...
67,2020-02-11,봉준호 감독의 기생충이 아카데미 시상식에서 작품상 등 4개 부문을 휩쓸자 중국 언론...
68,2020-02-11,중국 매체 시나닷컴은 11일 봉준호 감독이 92회 아카데미 시상식에서 작품상은 물...


In [270]:
btsnews['article_original'][69]

'이해찬 더불어민주당 대표는 12일 “앞으로도 제2의 제3의 ‘기생충’ 같은 영화와 방탄소년단BTS 같은 뮤지션이 한국 문화 영역을 꽃피울 수 있도록 노력하겠다”고 밝혔다.  이 대표는 이날 오전 서울 여의도 국회에서 열린 최고위원회의에서 “봉준호 감독이 만든 ‘기생충’이 아카데미 영화제에서 4개 부문을 석권했다”고 축하하며 이같이 말했다.  그는 “한국은 높은 문화의 힘을 가진 나라가 돼가고 있다”며 “방탄소년단의 음악은 전 세계 청년의 마음을 울리고 한류는 세계로 뻗어가고 있다”고 말했다.  이어 “문화의 힘은 창의력에서 나오고 이는 시민을 보장하는 민주주의에 있다”며 “민주당과 문재인 정부는 지원은 하되 간섭을 하지 않는다는 원칙으로 자유롭고 창의로운 문화 창달을 지원하겠다”고 약속했다.  또 이 대표는 “신종 코로나 방역과 치료만큼 중요한 것이 경제 영향을 최소화하는 것”이라며 중소기업과 상공인 피해를 최소화하기 위해 세금 지원대책을 마련하겠다고 밝혔다.  그는 “당정이 함께 경영과 금융 판매지원까지 중소기업과 소상공인 지원 대책을 마련해 정부가 조금 전 발표했다”며 “현장과 지속 소통해 세금 문제까지 지원 대책을 마련하겠다”고 밝혔다.  전날 1차로 마무리된 인재영입과 관련해선 “중요한 목표는 민주당이 국민을 위해 더 잘 일할 수 있는 역량을 갖추도록 한 것”이라며 “일하는 국회와 책임있는 정부를 만들도록 노력하는 민주당이 되도록 하겠다”고 말했다.'

### 요약봇 활성화 (KOBERT + summarizer)

In [11]:
# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/kobert')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
custom_model = AutoModel.from_pretrained('monologg/kobert', config=custom_config)

summarize_model = Summarizer(custom_model=custom_model, custom_tokenizer = custom_tokenizer)
summarize_model2 = Summarizer()  

In [12]:
summarized_list = []
for i in range(len(btsnews['article_original'])):
#     summarized_list.append(summarize_model (btsdf['article_original'][i], ratio = 0.2))
    summarized_list.append(summarize_model(btsnews['article_original'][i], ratio= 0.2))

In [13]:
summarized_list

['12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운트다운과 함께 떨어지는 행사을 보기 위해 몰려든 사람들로 이곳은 오전부터 부산스러웠다. BTS는 밤 10시 38분부터 8분간 열정적인 춤과 노래로 시선을 사로잡았다. BTS는 2017년 사전 녹화를 통해 할리우드 무대에 출연했지만 타임스스퀘어 무대에 직접 선 것은 이번이 처음이다.',
 '12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상적인 영향력을 행사하면서 음악산업의 변화를 이끈 아티스트 톱 10을 선정해 발표했다. CNN은 BTS에 대해 K팝을 미국에 대중화시켰고 K팝을 주류음악maistream으로 이끌었다며 이들의 음악이 2010년대 말미에 부상한만큼 미국인들 사이에서 2020년대 K팝은 더 저변을 넓힐 수 있을 것이라고 말했다.',
 '특히 지난해 전 세계적인 신드롬을 일으켰던 방탄소년단BTS의 새앨범과 투어가 시작되고 트와이스 블랙핑크 등도 활동에 박차를 가한다. 특히 포스트 말론 등 주요 출연진만 선 카운트다운 무대에 함께해 방탄소년단의 국제적 위상을 보여줬다는 평가다. 트와이스는 최근 빌보드 인터뷰에서 2020년 활동에 대해 콘서트 투어 새 앨범 등 많은 즐거운 일들을 계획하고 있다고 소개했다.',
 'K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정해 홍보하고 있다. 멀리서 한국을 찾은 외국인 관광객들이 방탄소년단BTS 트와이스 등 스타들의 흔적을 찾아다닐 수 있도록 추천한다는 계획이다. 잔디밭 뒤로 한강과 반포대교가 보이는 낯익은 한강공원의 풍경은 영상 속에서 방탄소년단 멤버들이 옹기종기 모여 배달된 피자와 치킨을 먹던 바로 그 장소다.',
 '골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 매체와 인터뷰 중 방탄소년단BTS을 언급하며 한국을 ‘멋진 아티스트들의 나라’라고 표현했다. 봉 감독은 제가 비록 골든글로브에 와 있긴 하지만 BTS가 누리는 파워와 힘은

https://stackoverflow.com/questions/47159996/how-to-group-text-data-based-on-document-similarity

In [271]:
btsnews['summary'] = summarized_list

In [273]:
btsnews.to_csv('./data/btsnews.csv', encoding = 'utf-8-sig', index = False)

### 토크나이징

In [14]:
from nltk.corpus import stopwords 

In [274]:
with open ("./data/stopwords.txt", "r", encoding = 'utf-8') as myfile:
    data=myfile.read()
btsnews = pd.read_csv('./data/btsnews.csv', encoding = 'utf-8-sig')

In [167]:
data
stop_words=data.split('\n')

In [168]:
from konlpy.tag import Kkma, Okt, Mecab  
kkma=Kkma()
okt =Okt()
# tkexample = okt.nouns(btsnews['article_original'][90])

In [169]:
# 불용어 처리를 거친 okt.nouns function
def stopwords(df):
    result = []
    
    for w in okt.nouns(df):
        if w not in stop_words:
            result.append(w)
    return result

In [170]:
stopwords(btsnews['article_original'][4])

['골든글로브',
 '시상식',
 '한국',
 '최초',
 '외국어',
 '영화상',
 '수상한',
 '기생충',
 '봉준호',
 '감독',
 '매체',
 '인터뷰',
 '중',
 '방탄소년단',
 '언급',
 '한국',
 '아티스트',
 '나라',
 '표현',
 '봉준호',
 '감독',
 '오전',
 '한국',
 '미국',
 '로스앤젤레스',
 '베버',
 '리',
 '힐튼',
 '호텔',
 '골든글로브',
 '시상식',
 '레드카펫',
 '행사',
 '캐나다',
 '인터뷰',
 '중',
 '한국',
 '콘텐츠',
 '영향력',
 '대해',
 '봉',
 '감독',
 '골든글로브',
 '파워',
 '힘',
 '배',
 '며',
 '아티스트',
 '수',
 '나라',
 '감정',
 '다이나믹',
 '나라',
 '한편',
 '날',
 '봉준호',
 '감독',
 '영화',
 '기생충',
 '한국',
 '영화',
 '최초',
 '골든글로브상',
 '쾌거',
 '골든글로브',
 '아카데미',
 '오스카',
 '전초전',
 '만큼',
 '내달',
 '아카데미',
 '시상식',
 '수상',
 '가능성',
 '평가',
 '기생충',
 '각본',
 '상과',
 '감독',
 '후보',
 '선정',
 '수상',
 '날',
 '각본',
 '원스',
 '어폰',
 '타임',
 '인',
 '할리우드',
 '쿠엔틴',
 '타란티노',
 '감독',
 '감독',
 '샘',
 '데스',
 '감독']

In [275]:
btsnews['tokenized_sents'] = btsnews.apply(lambda row: stopwords(row['article_original']), axis=1)

In [276]:
btsnews['tokenized_sents'][3]

['팝',
 '대한',
 '관심',
 '전',
 '세계',
 '서울시',
 '팝',
 '명소',
 '선정',
 '홍보',
 '멀리',
 '한국',
 '외국인',
 '관광객',
 '방탄소년단',
 '트와이스',
 '스타',
 '흔적',
 '수',
 '추천',
 '계획',
 '서울시',
 '서울',
 '속',
 '팝',
 '명소',
 '곳',
 '추천',
 '서울',
 '글로벌',
 '인기',
 '스타',
 '거듭',
 '흔적',
 '서울',
 '소개',
 '영상',
 '멤버',
 '산책',
 '땐',
 '하늘',
 '위',
 '서울',
 '하늘',
 '위',
 '길',
 '서울',
 '보행',
 '곳곳',
 '식물',
 '도심',
 '속',
 '공중',
 '정원',
 '곳',
 '엑소',
 '첸',
 '숲',
 '수호',
 '숲',
 '트와이스',
 '사나',
 '숲',
 '팝',
 '스타',
 '이름',
 '숲',
 '조성',
 '기도',
 '잔디밭',
 '뒤',
 '한강',
 '반포대교',
 '낯',
 '한강',
 '공원',
 '풍경',
 '영상',
 '속',
 '방탄소년단',
 '멤버',
 '옹기종기',
 '배달',
 '피자',
 '치킨',
 '장소',
 '반포',
 '한강',
 '공원',
 '봄',
 '서래섬',
 '유채꽃',
 '만발',
 '가을',
 '공원',
 '산책로',
 '대숲',
 '우거지',
 '공원',
 '시립',
 '미술관',
 '스타',
 '곳',
 '지난',
 '진행',
 '데이비드',
 '호크',
 '전',
 '신세경',
 '차',
 '은우',
 '김동률',
 '아이비',
 '스타',
 '방문',
 '전시',
 '총',
 '관람',
 '올해',
 '국내',
 '미술',
 '전시',
 '중',
 '가장',
 '화제',
 '입구',
 '스산',
 '분위기',
 '망우동',
 '폐장',
 '놀이',
 '공원',
 '랜드',
 '영화',
 '드라마',
 '촬영',
 '지로',
 '듯',
 '빛바랜',
 '놀이기구',
 '그대로',
 '남아'

### 문서 유사도

In [174]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [175]:
article_data =btsnews['tokenized_sents'].astype(str)
tfidf_matrix = tfidf.fit_transform(article_data)

In [176]:
# 코사인유사도 공식
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [277]:
indices = pd.Series(btsnews.index, index=btsnews['tokenized_sents']).drop_duplicates()
print(indices.head())

tokenized_sents
[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스퀘어, 무대, 선, 저녁, 타임스스퀘어, 볼드롭, 대형, 크리스털, 볼, 신년, 카운트다운, 행, 보기, 위해, 사람, 곳, 오전, 매년, 날, 최소, 곳, 올해, 때문, 열기, 더욱, 뉴욕, 경찰, 날, 인파, 추정, 한국, 가수, 타임스스퀘어, 새해, 맞이, 무대, 오른, 건, 싸이, 두, 번, 밤, 열정, 춤, 노래, 시선, 전, 인파, 싸이, 강남스타일, 말춤, 날, 타임스스퀘어, 전, 세계, 아미, 팬, 위, 시, 야광, 봉, 한국어, 떼창, 선사, 공연, 뒤, 리더, 살, 홀로, 집, 영화, 광경, 눈앞, 감격, 멤버, 목소리, 해피, 뉴, 공연, 미국, 최대, 새해, 맞이, 라이브, 쇼, 방송, 뉴, 어스, 로킹, 이브, 통, ...]                         0
[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스트, 톱, 선정, 발표, 대해, 팝, 미국, 대중화, 팝, 주류, 음악, 음악, 말미, 부상, 만큼, 미국, 사이, 팝, 더, 저변, 수, 말, 음악, 전혀, 보지, 빅, 팬, 중, 명일, 수, 쪽, 영향력, 부정, 수, 말, 러브, 유, 셀프, 전, 티어, 팝, 앨범, 최초, 미국, 빌보드, 차트, 위, 차지, 이후, 해, 차례, 빌보드, 차트, 위, 오른, 사실, 강조, 미국, 포함, 전, 세계, 장의, 앨범, 판매, 기록, 소개, 원더걸스, 노바디, 싸이, 강남, 스타일, 빌보드, 차트, 성과, 사실, 팝, 늘, 아시아, 인기, 미국, 시장, 위해, 분투, 압도, 평가, 한편, 방탄소년단, 날, 밤, 뉴욕, ...]                                        1
[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세계, 신드롬, 방탄소년단, 앨범, 투어, 시작, 트와이스, 블랙, 핑크, 활동, 박차, 가장, 관심사, 세계

### 문사유사도 측정 함수 정의

In [278]:
# 코사인 유사도를 통한 유사한 기사 뽑아내는 함수
def getcosine (df):
    article_data =df.astype(str)
    tfidf_matrix = tfidf.fit_transform(article_data)
    # 코사인유사도 공식
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

# 각 인덱스에 해당하는 기사와 유사한 기사들의 인덱스 추출
def similarity(index,df):
    result= []
    cosine_sim=getcosine(df)
 
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    article_indices = [i[0] for i in sim_scores]
    # 날짜가 너무 퍼지지 않게 Index 범위 조정
    for n in article_indices:
        if n < (int(index)+30) and n > (int(index)-30):
            result.append(n)
    return result

# 해당 컬럼에 해당하는 모든 열에 대하여 수행
def similarlist (df):
    new_list = []
    for i in range(len(df)):
        new_list.append(similarity(i,df))
    return new_list

In [211]:
similarity(15)

[10, 9]

In [279]:
btsnews['article_original'][9]

' 소속사 빅히트엔터테인먼트는 8일 오전 0시 팬 커뮤니티 ‘BTS 위버스’ 공지를 통해 “오는 2월21일 BTS 맵 오브 더 솔  7MAP OF THE SOUL  7이 발매된다”고 밝혔다.  방탄소년단 리더 RM은 지난해 말 미국 연예매체 버라이어티 주최 시상식에서 “좋은 소식은 우리가 새로운 음악 새로운 앨범을 작업하고 있다는 것”이라며 “가까운 장래에 여러분과 공유할 것이 기대된다”고 말한 바 있다.  방탄소년단의 새 앨범은 지난해 4월 ‘맵 오브 더 솔’ 연작의 첫 앨범인 맵 오브 더 솔  페르소나MAP OF THE SOUL  PERSONA 이후 약 10개월 만이다. 롤링스톤과 빌보드 등 외신도 신보 발매 소식을 보도하고 영국 오피셜차트도 사회관계망서비스SNS 계정에서 신보 예고를 전하는 등 세계적으로 관심이 집중됐다.  전작인 ‘러브 유어셀프’가 스스로 외우는 주문 혹은 세상을 향한 외침이었다면 ‘맵 오브 더 솔’은 내면 속 영혼의 지도를 찾는 데 주력한다. 팬들이 주는 사랑의 힘의 근원과 그늘 또 그 힘이 이끄는 내일에 대한 이야기를 담았다. 국내외 아미방탄소년단 팬들은 맵 오브 더 솔  페르소나에 이어 이번 앨범에는 또 어떤 메시지를 담겼을지 기대감을 공유했다.  방탄소년단이 이번 앨범으로 어떤 새로운 기록을 탄생시킬지에도 이목이 쏠린다. 맵 오브 더 솔 페르소나는 369만장 판매고를 올리며 가온차트가 2011년 집계를 시작한 이래 최다 판매량을 기록했다. 또 발매 직후 빌보드 메인 앨범 차트인 ‘빌보드 200’ 정상에 진입하며 국외에서도 각종 굵직한 기록을 남겼다.  방탄소년단은 앞서 러브 유어셀프 전 티어LOVE YOURSELF 轉 Tear와 러브 유어셀프 결 앤서LOVE YOURSELF 結 Aswer까지 3장 앨범을 11개월 사이 ‘빌보드 200’ 정상에 올려놨다.'

In [280]:
btsnews['common'] = similarlist(btsnews['article_original'])

In [281]:
# List 로 된 Similar Article String 타입으로
btsnews['sim'] = [', '.join(map(str, l)) for l in btsnews['common']]

In [282]:
btsnews

,date,article_original,summary,tokenized_sents,common,sim
0,2020-01-02,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","[1, 2]","1, 2"
1,2020-01-02,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상적...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",[0],0
2,2020-01-03,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,특히 지난해 전 세계적인 신드롬을 일으켰던 방탄소년단BTS의 새앨범과 투어가 시작되...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","[9, 19, 11, 0]","9, 19, 11, 0"
3,2020-01-03,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,"[팝, 대한, 관심, 전, 세계, 서울시, 팝, 명소, 선정, 홍보, 멀리, 한국,...",[],
4,2020-01-06,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,"[골든글로브, 시상식, 한국, 최초, 외국어, 영화상, 수상한, 기생충, 봉준호, ...",[],
...,...,...,...,...,...,...
853,2020-12-26,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 이젠...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","[852, 842, 830]","852, 842, 830"
854,2020-12-26,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...",[834],834
855,2020-12-29,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,봉준호 감독은 ‘기생충’으로 미 아카데미 4관왕에 오르며 한국 영화사를 다시 썼다....,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","[847, 848]","847, 848"
856,2020-12-30,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,"[박영선, 중소, 벤처기업, 부, 장관, 발표, 신년사, 최선, 더, 힘, 내, 송...",[],


### 데이터프레임 최종 정제

In [283]:
# 존재했던 데이터프레임에서 유사도가 겹치지 않는 기사들을 drop, 그리고 재정렬 
# 재정렬하면 유사도 인덱스또한 변동 사항이 때문에 다시 받아야한다
bts = btsnews.drop(columns = 'common')
bts = bts.replace('', np.nan)
bts = bts.dropna()
bts = bts.drop(columns = 'sim')
bts = bts.reset_index (drop=True)

In [284]:
bts['sim'] = similarlist(bts['tokenized_sents'])
bts['sim'] = [', '.join(map(str, l)) for l in bts['sim']]
bts

,date,article_original,summary,tokenized_sents,sim
0,2020-01-02,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","1, 2"
1,2020-01-02,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상적...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",0
2,2020-01-03,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,특히 지난해 전 세계적인 신드롬을 일으켰던 방탄소년단BTS의 새앨범과 투어가 시작되...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","6, 8, 26"
3,2020-01-07,‘‘BTS’ 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트...,‘‘BTS’ 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트...,"[상표권, 그룹, 방탄소년단, 소속사, 빅히트, 엔터테인먼트, 이하, 빅히트, 분쟁...",4
4,2020-01-07,방탄소년단BTS 소속사 빅히트엔터테인먼트와 ‘비티에스’BTS 상표권을 두고 분쟁 중...,방탄소년단BTS 소속사 빅히트엔터테인먼트와 ‘비티에스’BTS 상표권을 두고 분쟁 중...,"[방탄소년단, 소속사, 빅히트, 엔터테인먼트, 비, 티에스, 상표권, 분쟁, 중, ...",3
...,...,...,...,...,...
746,2020-12-26,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대에...,2017년 처음 미국 무대에 선 뒤 지난 3년 동안 빌보드와 아메리칸 뮤직 어워드에...,"[방탄소년단, 전세계, 인기, 이제, 처음, 미국, 무대, 선, 뒤, 지난, 빌보드...","748, 747, 728, 730, 732"
747,2020-12-26,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 이젠...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","746, 725, 737"
748,2020-12-26,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...","746, 747"
749,2020-12-29,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,봉준호 감독은 ‘기생충’으로 미 아카데미 4관왕에 오르며 한국 영화사를 다시 썼다....,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","741, 742, 729"


In [285]:
# 유사도 인덱스를 더한 각 행의 total 값을 뽑아준다.
simcount = pd.Series(bts['sim'].str.count('\w+'), name = 'simcount')
bts = pd.concat([bts, simcount], axis=1)

In [286]:
# 내 마음속에 저장
bts.to_csv ('./data/btsfinal.csv', index = False, encoding = 'utf-8-sig')
# 쓸데없는 (유사도가 겹치지않는) 기사들을 제거 한 최종 데이터 프레임
bts

,date,article_original,summary,tokenized_sents,sim,simcount
0,2020-01-02,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","1, 2",2
1,2020-01-02,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상적...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",0,1
2,2020-01-03,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,특히 지난해 전 세계적인 신드롬을 일으켰던 방탄소년단BTS의 새앨범과 투어가 시작되...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","6, 8, 26",3
3,2020-01-07,‘‘BTS’ 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트...,‘‘BTS’ 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트...,"[상표권, 그룹, 방탄소년단, 소속사, 빅히트, 엔터테인먼트, 이하, 빅히트, 분쟁...",4,1
4,2020-01-07,방탄소년단BTS 소속사 빅히트엔터테인먼트와 ‘비티에스’BTS 상표권을 두고 분쟁 중...,방탄소년단BTS 소속사 빅히트엔터테인먼트와 ‘비티에스’BTS 상표권을 두고 분쟁 중...,"[방탄소년단, 소속사, 빅히트, 엔터테인먼트, 비, 티에스, 상표권, 분쟁, 중, ...",3,1
...,...,...,...,...,...,...
746,2020-12-26,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대에...,2017년 처음 미국 무대에 선 뒤 지난 3년 동안 빌보드와 아메리칸 뮤직 어워드에...,"[방탄소년단, 전세계, 인기, 이제, 처음, 미국, 무대, 선, 뒤, 지난, 빌보드...","748, 747, 728, 730, 732",5
747,2020-12-26,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 이젠...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","746, 725, 737",3
748,2020-12-26,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...","746, 747",2
749,2020-12-29,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,봉준호 감독은 ‘기생충’으로 미 아카데미 4관왕에 오르며 한국 영화사를 다시 썼다....,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","741, 742, 729",3


In [298]:
# 기사가 8개 이상 겹치는 기사들만 출력 (최대 핫이슈로 예상)
btsfine = bts[bts['simcount'] > 7]
btsfine

,date,article_original,summary,tokenized_sents,sim,simcount
82,2020-02-24,“데뷔 7년 동안 방황한 적도 있었고 내면의 두려움과 그림자가 커졌던 시기도 있었지...,“데뷔 7년 동안 방황한 적도 있었고 내면의 두려움과 그림자가 커졌던 시기도 있었지...,"[데뷔, 방황, 적도, 내면, 두려움, 그림자, 시기, 이제, 무게중심, 법, 알,...","84, 81, 83, 78, 70, 60, 69, 61",8
84,2020-02-25,“7년이라는 시간을 함께 보내면서 휘청일 때도 중심을 못 잡고 방황할 때도 있었어요...,“7년이라는 시간을 함께 보내면서 휘청일 때도 중심을 못 잡고 방황할 때도 있었어요...,"[중심, 못, 잡고, 방황, 내면, 그림자, 마음, 이제, 정도, 무게, 중심, 잡...","81, 82, 69, 83, 60, 72, 61, 70",8
181,2020-05-18,K팝 그룹 방탄소년단 멤버 정국과 아스트로 멤버 차은우 세븐틴 멤버 민규 NCT12...,K팝 그룹 방탄소년단 멤버 정국과 아스트로 멤버 차은우 세븐틴 멤버 민규 NCT12...,"[팝, 그룹, 방탄소년단, 멤버, 정국, 아스트로, 멤버, 차, 은우, 세븐, 틴,...","185, 184, 182, 183, 180, 186, 198, 152",8
195,2020-05-31,방탄소년단BTS 멤버 슈가의 개인 믹스테이프비정규 무료음반 수록곡에 미국 사이비 종...,방탄소년단BTS 멤버 슈가의 개인 믹스테이프비정규 무료음반 수록곡에 미국 사이비 종...,"[방탄소년단, 멤버, 슈가, 개인, 믹스테이프, 비, 정규, 무료, 음반, 수록, ...","196, 202, 198, 200, 205, 216, 207, 204",8
196,2020-05-31,방탄소년단BTS 슈가의 믹스테이프에 미국 사이비 종교 교주 제임스 워런 짐 존스의 ...,방탄소년단BTS 슈가의 믹스테이프에 미국 사이비 종교 교주 제임스 워런 짐 존스의 ...,"[방탄소년단, 슈가, 믹스테이프, 미국, 사이비, 종교, 교주, 제임스, 워런, 짐...","198, 195, 200, 202, 205, 204, 216, 207",8
...,...,...,...,...,...,...
682,2020-11-26,방탄소년단BTS·사진이 내년 1월 31일 열리는 제63회 그래미 어워드 후보에 올랐...,방탄소년단BTS·사진이 내년 1월 31일 열리는 제63회 그래미 어워드 후보에 올랐...,"[방탄소년단, 사진, 내년, 그래미, 어워드, 후보, 미국, 레, 코딩, 아카데미,...","677, 684, 680, 676, 675, 681, 679, 674, 678",9
684,2020-11-26,방탄소년단이 지난 24일현지 시각 제63회 그래미 어워즈의 ‘베스트 팝 듀오·그룹...,방탄소년단이 지난 24일현지 시각 제63회 그래미 어워즈의 ‘베스트 팝 듀오·그룹 ...,"[방탄소년단, 지난, 그래미, 어워즈, 베스트, 팝, 듀오, 그룹, 퍼포먼스, 후보...","679, 682, 680, 677, 681, 675, 678, 676, 674",9
698,2020-12-01,그룹 방탄소년단BTS의 신곡 ‘라이프 고스 온’이 미국 빌보드 메인 싱글 차트 정...,그룹 방탄소년단BTS의 신곡 ‘라이프 고스 온’이 미국 빌보드 메인 싱글 차트 정상...,"[그룹, 방탄소년단, 신곡, 라이프, 고스, 온, 미국, 빌보드, 메인, 싱글, 차...","704, 700, 696, 707, 699, 708, 695, 710",8
703,2020-12-01,30일현지시간 방탄소년단의 한국어 신곡 ‘라이프 고스 온Life Goes O’이 ...,30일현지시간 방탄소년단의 한국어 신곡 ‘라이프 고스 온Life Goes O’이 미...,"[방탄소년단, 한국어, 신곡, 라이프, 고스, 온, 미국, 빌보드, 메인, 싱글, ...","707, 698, 708, 696, 704, 700, 699, 695",8


In [300]:
btsfine.summary[682]

'방탄소년단BTS·사진이 내년 1월 31일 열리는 제63회 그래미 어워드 후보에 올랐다. 2018년 5월 슈가가 밝힌 빌보드 양대 차트 1위 그래미 시상식 참석 스타디움 투어 등의 목표가 전부 이뤄지면서 그래미 수상과 단독 무대는 방탄소년단에게 유일하게 남은 목표였다. 외신들은 방탄소년단이 그래미상 후보에 오르자 “그래미의 역사를 다시 썼다”며 찬사를 보내면서도 ‘올해의 레코드’와 ‘올해의 노래’ 등 주요 부문 후보에서 빠진 것을 지적하기도 했다.'